<a href="https://colab.research.google.com/github/Gibbons-Lab/isb_course_2024/blob/main/16S_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🦠 Amplicon Sequencing Data Analysis with QIIME 2

### 🔗 Important Links
- [Course Github](https://github.com/Gibbons-Lab/isb_course_2024): Github repository for the 2024 ISB Virtual Microbiome Series

    - [Day 1 Slides](https://gibbons-lab.github.io/isb_course_2024/16S): Presentation slides that accompany this notebook

    - [Day 1 Solutions](https://github.com/Gibbons-Lab/isb_course_2024/16S_solutions.ipynb): Pre-run notebook 

    - [Treasure chest](https://github.com/Gibbons-Lab/isb_course_2024/tree/main/treasure_chest): All files generated in the notebooks
    
- [QIIME 2 view](http://view.qiime2.org): A browser-based interface for viewing QIIME artifacts and visualizations (tables and plots)
- [QIIME 2 plugins](https://docs.qiime2.org/2024.5/plugins/): more information on the QIIME 2 plugins we're using today

### 🛑 STOP! Before you run anything ...

1. Save your own local copy of this notebook by using `File > Save a copy in Drive`. At some point you may be prompted to trust the notebook. We promise that it is safe 🤞

2. Keep in mind, **the code in this notebook must be run IN ORDER** If you get lost or encounter errors, all outputs can be found in `materials/treasure chest` or on the link above.

**Disclaimer:** The Google Colab notebook environment will interpret any command as Python code by default. If we want to run bash commands we will have to prefix them by `!`. So any command you see with a leading `!` is a bash command and if you wanted to run it in your terminal you would omit the `!`. For example, if in the Colab notebook you ran `!wget` you would just run `wget` in your terminal.

---
# Setup

QIIME 2 is usually installed by following the [official installation instructions](https://docs.qiime2.org/2024.5/install/). However, because we are using Google Colab and there are some caveats to using conda here, we will have to hack around the installation a little bit. But no worries, we provide a setup script below which does all this work for us. 😌

So...let's start by pulling a local copy of the project repository down from GitHub. This repository, called `materials`, contains all the relevant data and other resources we'll need for this course. 

In [ ]:
!git clone https://github.com/gibbons-lab/isb_course_2024 materials

To view the directory, you can click on the folder 📁 icon on the left. To run code from this directory, we will navigate to it via command line:

In [ ]:
%cd materials

Notice here we use ```%``` instead of ```!``` to run our command line function. This makes the path change to our directory _permanent_. Using the ```!``` operator only switches the interpreter to expect command line prompts temporarily.




Now that we have all our materials, we're _almost_ ready to get started - but not quite. Remember QIIME? We'll need to install that before getting into the actual analysis. Don't worry - this will only set up in the Colab notebook, not on your local machine.

**Let's run the following cell, to install and setup QIIME2**

In [ ]:
%run setup_qiime2

⬆️ This will take some time (usually 10 to 15 minutes), so we'll switch back over to the [presentation](https://gibbons-lab.github.io/isb_course_2024/16S) while we wait.

If you want to learn more about QIIME2, we recommend you check out the [documentation](https://docs.qiime2.org/). This will also explain how to install QIIME2 on your local machine 🖥

---
# Data Import

QIIME2 can take us from raw sequences to ecological insight!
![our workflow](https://github.com/Gibbons-Lab/isb_course_2024/raw/main/docs/16S/assets/steps.png)
But we first need to get our raw data into QIIME2. To do this, we will need:
1. **Raw sequencing files** (.fastq): FASTQ files contain both sequence and quality scores, as opposed to FASTA, which only have sequence. We want those quality scores so that we can trim or discard low-quality reads.

2. **Metadata file** (.tsv): This file contains the sample id, along with all non-sequencing data for each sample. Here, our metadata includes disease status, age, sex, BMI, geographical location, and prescription drug use, as well as some information on diet and GI symptoms.

3. **Manifest file** (.tsv): This file contains the sample id and the path to its sequencing file. This is how QIIME matches a sequence to its correct metadata. 📝

Let's begin by taking a look at our data. In the `data` folder, you'll find 10 FASTQ files, a file manifest, and a metadata file.

If we read in `manifest.tsv`, we can see that it is a table containing the _name_ and _filepath_ of all our samples.

In [ ]:
import pandas as pd
manifest = pd.read_csv('data/manifest.tsv', sep = '\t')
manifest

We can also check out `metadata.tsv`, which will give us more context on our samples 🔬

In [ ]:
metadata = pd.read_csv('data/metadata.tsv', sep='\t')
metadata

Looks good, all 10 FASTQ files are accounted for, five healthy and five with Parkinson's Disease. We can use the manifest file to import our files into QIIME.

## FASTQ to Artifact

To use sequencing data in QIIME, we first need to turn the FASTQ files containing our data into QIIME artifacts. Using the manifest we just checked out, let's run our first command:

-- as a reminder, adding ```!``` before the command tells the notebook this is a **bash** command, rather than python.

In [ ]:
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path data/manifest.tsv \
  --output-path sequences.qza \
  --input-format SingleEndFastqManifestPhred33V2

Let's take a look a the command. QIIME commands take following format:

```
qiime plugin action --i-argument1 ... --o-argument2 ...
```
In the previous command, we are calling the ```tools``` plugin within QIIME2 to import our data. The following arguments designate where the manifest is, as well as where the output should be saved. We also tell QIIME what sort of input to expect.

Argument types usually begin with a letter denoting their meaning:

- `--i-...` = input files
- `--o-...` = output files
- `--p-...` = parameters
- `--m-...` = metadata

## Artifact to Visualization 🔎

Before we move on, let's use QIIME to visualize our sequencing data.

In [ ]:
!qiime demux summarize \
--i-data sequences.qza \
--o-visualization qualities.qzv

.qzv files like the one we just produced are visualization. You can view the plot by **downloading the file and opening it using http://view.qiime2.org**. To download the file click on the folder symbol to the left, open the `materials` folder, and choose download from the dot menu next to the `qualities.qzv` file.

---
# Quality Filtering: From Sequence to ASV


Before we can use our sequencing data, we need to "denoise" it. To do this, we'll use a *plugin* called ```DADA2```. This involves a few things:

1. filter and trim the reads
2. find the most likely set of unique sequences in the sample (ASVs)
3. remove chimeras
4. count the abundances of each ASV


This command will take a little time - let's run it, and head back to the presentation to discuss what's happening.

In [ ]:
!qiime dada2 denoise-single \
    --i-demultiplexed-seqs sequences.qza \
    --p-trunc-len 150 \
    --p-n-threads 2 \
    --output-dir dada --verbose


If this step takes too long or fails, you can also copy the results from the treasure chest with the following command.

In [ ]:
# obscure magic that will only copy if the previous command failed
![ -d dada ] || cp -r treasure_chest/dada .

If we open up the `dada` folder 📁, we will see several outputs.
- `representative_sequences.qza` is our *ASV abundance table* 🧬
- `denoising-stats.qza` contains the *denoising statistics* 📊

We'll want to look at the denoising statistics to make sure the denoising went well. It is normal to lose between 5-25% of reads to chimeras, but major loss could be due to running too many PCR cycles.

As mentioned before, we can't view QIIME *artifacts* directly, but we can convert them to *visualizations* using the action `tabulate` from the plugin `metadata`.

In [ ]:
!qiime metadata tabulate \
    --m-input-file dada/denoising_stats.qza \
    --o-visualization dada/denoising-stats.qzv

Like before, we can download the .qzv file and visualize the results using the [QIIME2 Viewer](https://view.qiime2.org/).

It's important to understand what this output tells us. For instance, what percent of reads in our data pass the filtering step? What percent of reads were non-chimeric? Differences in these metrics between samples can affect diversity metrics.

For more information on paired-end quality filtering, check out the [2023 course](https://github.com/Gibbons-Lab/isb_course_2023)! 

---
# Phylogenetics

Now that we have an ASV table, we can calculate diversity metrics! But first, head back to the [presentation](https://gibbons-lab.github.io/isb_course_2024/16S) for an introduction to diversity!

Note: we're getting into analyses that some may prefer to perform in RStudio or Jupyter Notebooks, without the use of the QIIME framework. For example, diversity analyses can be performed using `scikit-bio` in Python or `vegan` in R. In fact, QIIME2 is using `scikit-bio` and `vegan` under the hood for its diversity calculations!

### From ASVs to Phylogenetic Trees with QIIME2 `phylogeny`

Some *diversity metrics* are calculated using **phylogenetic distance**, so let's start by building a phylogenetic tree for our sequences using the following command. This time, we call the `phylogeny` plugin in QIIME2.

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences dada/representative_sequences.qza \
    --output-dir tree

We can create a visualization for the tree using the `empress` QIIME 2 plugin. See [documentation](https://github.com/biocore/empress) for more details.

In [ ]:
!qiime empress tree-plot \
    --i-tree tree/rooted_tree.qza \
    --o-visualization tree/empress.qzv

---
# Diversity

## Part 1: The Basics

The QIIME2 `diversity` plugin has an action called `core-metrics-phylogenetic`, which calculates _several_ phylogenetic and non-phylogenetic diversity metrics.

We can use our table and tree to calculate several diversity metrics. To account for variations in **sampling depth**, we'll provide QIIME2 with a cutoff at which we will **rarefy** all our samples. Rarefaction  **randomly** selects sequences, so your results might look a little different from mine. We'll also pass in our metadata file, so we can keep track of which samples come from each group.

In [ ]:
!qiime diversity core-metrics-phylogenetic \
    --i-table dada/table.qza \
    --i-phylogeny tree/rooted_tree.qza \
    --p-sampling-depth 5000 \
    --m-metadata-file data/metadata.tsv \
    --output-dir diversity

If you open the `diversity` folder, you'll see that we calculated all of the diversity metrics I just mentioned. For more information on how these diversity metrics are calculated and interpreted, check out this QIIME [forum post](https://forum.qiime2.org/t/alpha-and-beta-diversity-explanations-and-commands/2282).

But what we're really interested in is whether diversity differs *significantly* between disease and healthy samples, or other covariates. Therefore, we will need to do statistical tests. 

Head back to the [slides](https://gibbons-lab.github.io/isb_course_2024/16S), and we'll go over some types of statistical tests, and when to use them.

### Alpha diversity: richness and evenness _within_ a sample

We get a bunch of outputs from the previous command - measures of both alpha and beta diversity. To start, let's use the Shannon vector in the output directory to create a visualization of alpha diversity across samples. Generally, healthy, long-living individuals have balanced diverse microbiomes. However, this isn't necessarily a direct indicator of health or disease. Let's see how it looks in our samples

In [ ]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity diversity/shannon_vector.qza \
    --m-metadata-file data/metadata.tsv \
    --o-visualization diversity/alpha_groups.qzv

In [ ]:
# do t-test or mann-whitney U?

Like before, we can download the visualization and open it with the [QIIME2 viewer](http://view.qiime2.org).

### Beta diversity: distance between samples

Let's visualize the beta diversity and see how they separate. For this we'll look at weighted UniFrac. This time, we'll have to download the file ⬅️

We can check for 'significant' separation between samples using PERMANOVA. We can do this with the diversity plugin in QIIME2.

In [ ]:
!qiime diversity adonis \
    --i-distance-matrix diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file data/metadata.tsv \
    --p-formula "parkinson_disease" \
    --p-n-jobs 2 \
    --o-visualization diversity/permanova.qzv

We already ran a PCoAs for each distance metric, and we can look at them if we download `weighted_unifrac_emperor.qzv`, `unweighted_unifrac_emperor.qzv`, `bray_curtis_emperor.qzv`, or `jaccard_emperor.qzv`

### Are these results informative? A discussion of sample size, effect size, and confounding

These results aren't very informative, and in some cases they are counterintuitive. There are several reasons this may be:

1. **Small sample size:** We are using a very small sample size (n = 10). Even if we could observe trends, we wouldn't expect statistical significance.

2. **Shannon diversity is a very coarse-grain metric:** Shannon Index attempts to summarize microbiome diversity (very complicated) into a single number.

3. **Strong confounders:** This data is from an observational study, were confounders were not controlled. Some confounders (like diet and drug exposure) are known to have strong effects on diversity, which may conceal smaller relationships between the microme and a variable of interest (like disease status).

In observational studies, it is common to **stratify** across confounders, or include them as **covariates** in regressions. It is recommended to have **at least 10 samples for every variable** in a multivariate regression like PERMANOVA, general linear model (GLM) or ___. So with a sample size n = 10, we can really only test 1 variable without breaking assumptions. 

**But what if we had a bigger sample size?** This would open the door to more types of analyses. We couldn't process the entire dataset on Colab due to limited computational power, but luckily, **we've pre-processed the entire dataset for you**!

## Part 2: Deconfounding

### Alpha Diversity Deconfounding: Stratification + Multivariate Regression

The full Hill-Burns 2017 dataset files are in the treasure chest. Let's load them into QIIME2 now.

In [ ]:
# Copy full dataset directory into your local directory
![ -d complete_data ] || cp -r treasure_chest/complete_data .

In [ ]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity complete_data/diversity/shannon_vector.qza \
    --m-metadata-file complete_data/data/metadata.tsv \
    --o-visualization complete_data/diversity/alpha_groups.qzv

In the original paper, the authors found that X, X, and X had significant, independent effects on microbiome composition. Can we recapitulate this finding? 

What happens if we __stratify__ by anticholinergic drug use? To split the data into different groups, we can use the QIIME2 `filter` action.

In [ ]:
# filter tables
!qiime filter xxxxx \
    --i-something 

Then we run the diversity functions separately for the AC and noAC groups

In [ ]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity complete_data/diversity/shannon_vector_AC.qza \
    --m-metadata-file complete_data/data/metadata.tsv \
    --o-visualization complete_data/diversity/alpha_groups_AC.qzv

!qiime diversity alpha-group-significance \
    --i-alpha-diversity complete_data/diversity/shannon_vector_noAC.qza \
    --m-metadata-file complete_data/data/metadata.tsv \
    --o-visualization complete_data/diversity/alpha_groups_noAC.qzv

Linear regression for Shannon diversity?

In [ ]:
!qiime

### PERMANOVA + PCoA for Beta diversity

Since we now have n = 300, we can probably do a PERMANOVA. But first, we need to make sure the variables we include have enough samples in each category.

In [ ]:
# get summaries of each variable we want to include in PERMANOVA

First, let's run ANOVA for just disease status so we can really see what adding covariates does to our results

In [ ]:
!qiime diversity adonis \
    --i-distance-matrix complete_data/diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file complete_data/data/metadata.tsv \
    --p-formula "parkinson_disease" \
    --p-n-jobs 2 \
    --o-visualization complete_data/diversity/permanova.qzv

We can also use PERMANOVA to identify confounders. PERMANOVA tells us how much variance in the community composition is explained by each variable. Common confounders include sex, age, BMI, diet, and antibiotic use. In the original study, authors identified that Parkinson's Disease medications were associated with different microbiome compositions. Let's take a look at these variables.

In [ ]:
!qiime diversity adonis \
    --i-distance-matrix complete_data/diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file complete_data/data/metadata.tsv \
    --p-formula "parkinson_disease + sex + age + location + p3m_antibiotics_bool" \
    --p-n-jobs 10 \
    --o-visualization complete_data/diversity/permanova_big.qzv

Before, we did not see a significant p-value for the effect of Parkinson's disease on beta diversity. However, when we add certain covariates, we might find that they were confounding a relationship.

However, most of our variance remains unexplained. Microbiome composition is affected by many things, and this is an uncontrolled cohort study, so we would not expect any single variable to explain most of the variance.

#### Dimensionality reduction

If we want to __visually__ show this separation between samples, we can't just plot the entire UniFrac distance matrix, because it has 100+ dimensions! Instead, we can use **dimensionality reduction** to "compress" our data into a few dimensions that explain most of the variance. There are several types of dimensionality reduction (like UMAP and tSNE), but the preferred method of dimensionality reduction for microbiome communities is Principal Coordinate Analysis (PCoA). This is because PCoA is linear, and thus preserves the global structure of the data and is reproducible.

We already ran a PCoAs for each distance metric, and we can look at them if we download `weighted_unifrac_emperor.qzv`, `unweighted_unifrac_emperor.qzv`, `bray_curtis_emperor.qzv`, or `jaccard_emperor.qzv`

---
# Taxonomic Classification

### From ASV to taxonomic count table

We can learn a lot from diversity metrics, alpha and beta. But to really dig into the data, we need to know what microbes are in each sample 🦠. To do this, we'll classify the reads in QIIME2 using a pre-trained Bayesian classifier. Several such classifiers are available at https://docs.qiime2.org/2024.5/data-resources/

From the QIIME2 plugin `feature-classifier`, we will use the action the action `classifiy-sklearn` which takes your ASVs and a pre-trained scikit-learn classifier as inputs. We are using a Naive Bayesian classifier specific to the 16S V4 region (515f-806r) because that is the region we sequence, but also it will make it easier to run MICOM tomorrow!

In [ ]:
!qiime feature-classifier classify-sklearn \
    --i-reads dada/representative_sequences.qza \
    --i-classifier ncbi-refseq-genus-515f-806r.qza \
    --p-n-jobs 2 \
    --o-classification taxa.qza

Now we've classified the reads, we can visualize the taxonomic breakdown of our samples.

In [ ]:
!qiime taxa barplot \
    --i-table dada/table.qza \
    --i-taxonomy taxa.qza \
    --m-metadata-file data/metadata.tsv \
    --o-visualization taxa_barplot.qzv

Now, we can use ```table.qza```, which contains our reads, and ```taxa.qza```, which contains taxonomic classifications for reads, and collapse the data onto the genus level.

In [ ]:
!qiime taxa collapse \
    --i-table dada/table.qza \
    --i-taxonomy taxa.qza \
    --p-level 6 \
    --o-collapsed-table genus.qza

### Exporting your taxa table

Taxonomic feature tables in QIIME2 can be exported as .biom files, which can then be converted to .tsv files if desired. This is important, because some analysis methods not available on QIIME2 may require .biom or .tsv formats.

We'll export this as a .tsv, which will be more usable for the next portion of the course that you'll see tomorrow.

In [ ]:
!qiime tools export \
    --input-path genus.qza \
    --output-path exported
    
!biom convert -i exported/feature-table.biom -o genus.tsv --to-tsv

Let's peek at the results 🔭

In [ ]:
abundances = pd.read_table("genus.tsv", skiprows=1, index_col=0)
abundances

This is easier to interpret by visualizing the results. We can use the file we just exported from QIIME2 to build a visualization using any tool we like, such as seaborn or plotnine.

Here is an example of building a visualization (a heatmap) in seaborn:

In [ ]:
import numpy as np
import seaborn as sns

In [ ]:
abund_to_plot = abundances
abund_to_plot.index = abund_to_plot.index.str.split(";").str[5]          # Use only the genus name
abund_to_plot = abund_to_plot[~abund_to_plot.index.isin(["g__", "__"])]  # remove unclassified genera
abund_to_plot = abund_to_plot.sample(50, axis=0)                         # use 50 random genera (rows)

# Let's do a centered log-ratio transform: log x_i - log mean(x)
transformed = abund_to_plot.apply(
    lambda xs: np.log(xs + 0.5) - np.log(xs.mean() + 0.5),
    axis=0)

sns.clustermap(transformed.T, cmap="magma", xticklabels=True, figsize=(16, 6))

Now, our data is starting to be interpretable. Each row is a sample, and each column is a bacterial genus. The table values are "counts", or the number of times a genus was detected in a certain sample. We can use relative abundance data to test hypotheses, but it requires special statistical methods because it is ___compositional___.

Let's discuss what we can do about that in the [presentation](https://gibbons-lab.github.io/isb_course_2024/16S).

---
# Differential Abundance Analysis

### "Normalization" + Statistical testing:
Center-log Ratio (CLR) Transformation, Wilcoxen test, multivariate regression, make correct assumptions, p-value correction

In [ ]:
# use pandas and mathematics?

I will have showed some CLR plots in slides maybe?

### Using ANCOM with QIIME2

In [ ]:
!qiime composition ancombc \
  --i-table genus.qza \
  --m-metadata-file data/metadata.tsv \
  --p-formula "parkinson_disease" \
  --o-differentials ancombc.qza

In [ ]:
!qiime composition da-barplot \
  --i-data ancombc.qza \
  --p-significance-threshold 0.01 \
  --o-visualization da_barplot.qzv

### More Robust Methods

radEmu

---
# Exercises:

### 1.   Plant a Tree

One visualization that we did not spend a lot of time on was the phylogentic tree of our ASVs. Let's change that! We have seen that there are genera that appear in multiple populations in the previous step. But are the organisms in that genus actually the same?

Let's annotate the tree with our taxonomic classifications and abundances. We will use the empress plugin again but this time with the `community-plot` option. I filled in a template of the command for you. Can you figure out what has to go in the empty spaces?

**QUESTIONS:**

1) Are some of the branch lengths on the tree longer than you would expect? Do you notice anything interesting or suspicious about the taxonomic identities of these branches?

2) Can you find examples of phyla that are polyphyletic (i.e. where clusters of ASVs from the same phylum are found in different locations on the tree, showing different commmon ancestors)? What about polyphyletic taxa at lower taxonomic levels, like at the family or genus levels? Why do you think these patterns exist?

In [ ]:
# This won't run until you fill in the [EMPTY] spots with the right files ;)

!qiime empress community-plot \
    --i-tree [EMPTY] \
    --i-feature-table dada/table.qza \
    --m-sample-metadata-file [EMPTY] \
    --m-feature-metadata-file taxa.qza \
    --o-visualization community-tree-viz.qzv

### 2.   Differential Abundance with Deconfounding